<a id='import-storage'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Storage </span> </p>

The Import Storage collects the following data upon import of a file:

- Current Period (year, month)
- Previous Period
- [Partitions](../DataModel/DataStructure)
- [RawVariables](../DataModel/DataStructure)
- [IfrsVariables](../DataModel/DataStructure)
- [Projection Configuration](../DataModel/DataStructure)
- [YieldCurve](../DataModel/DataStructure)
- [DataNodeParameters](../DataModel/DataStructure)
- [AoC Type](../DataModel/DataStructure)
- [AmountType](../DataModel/DataStructure)
- [EstimateType](../DataModel/DataStructure)
- [DataNodes](../DataModel/DataStructure)

The data required for the calculations is retrieved by queries of [parameters](../Utils/Queries#current-and-previous-parameters) and [transactional data](../Utils/Queries#data-variables) and stored in the storage to be available for the Import Scopes.
The calculation of a Best Estimate scenario is perfomerd using parameters from the data partition with Scenario Best Estimate (for both current and previous period values). For every scenario $S$ every query to parameters prioritize the data belonging to the partition with Scenario $S$ for the current period, if not present the value belonging to the Best Estimate scenario is used. Previous period parameter is always retrieved from the Best Estimate scenario. 

For Scenario calculations, the transactional data (RawVariable and IfrsVariable) stored in the ImportStorage is the union of the variables provided for the Scenario and the variables provided for the Best Estimate. In case of intersection betweeen the two sets, priority is given to the values belonging to the scenario. This allows users to input only the variables for scenario that differ from the Best Estimate scenario. For example, if a scenario affects only the Risk Adjustment variables, the nominal cashflow input might contain only the Risk Adjustment. If other variables are provided they are equally considered, stored in the data source and used in calculation. Note that if a scenario requires a variable provided under the Best Estimate scenario to have 0 value, then it must be input in the scenario file with 0 value. Omitting this variable from the input results in the calculation engine picking up the value provided in the Best Estimate scenario. This applies to nominal cash flow, actuals, and opening. 

Such storage is then passed to calculations defined in the corresponding Import Scopes:

- [1ImportScope-Identities](./1ImportScope-Identities)
- [2ImportScope-PresentValue](./2ImportScope-PresentValue)
- [3ImportScope-Actuals](./3ImportScope-Actuals)
- [4ImportScope-TechnicalMargin](./4ImportScope-TechnicalMargin)
- [5ImportScope-ToIfrsVar](./5ImportScope-ToIfrsVar)
- [6ImportScope-Compute](./6ImportScope-Compute)

In [0]:
#!import "../Utils/EqualityComparers"
#!import "../Utils/ImportCalculationMethods"
#!import "../Utils/Queries"

# Import Storage

In [0]:
public class ImportStorage
{   
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;
    
    // Constants
    private static int periodicityInMonths => 3; // Revisit
    
    //Format
    public string ImportFormat => args.ImportFormat; 
    
    //Time Periods 
    public (int Year, int Month) CurrentReportingPeriod => (args.Year, args.Month);
    public (int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic

    //Partitions
    public Guid PartitionByRn;
    public Guid TargetPartition;
    public Guid DefaultPartition;
    public Guid PreviousPeriodPartition;    

    //Projections
    private ProjectionConfiguration[] ProjectionConfiguration;
    
    //DataNodes
    public IDictionary<string, DataNodeData> DataNodeDataBySystemName { get; private set; }
    public IDictionary<ImportScope, HashSet<string>> DataNodesByImportScope { get; private set; }
    public IDictionary<string, ICollection<int?>> AccidentYearsByDataNode { get; private set; }
    //Variables
    public IDictionary<string, ICollection<RawVariable>> RawVariablesByImportIdentity { get; private set; }
    public IDictionary<string, ICollection<IfrsVariable>> IfrsVariablesByImportIdentity { get; private set; }
        
    //Parameters
    public Dictionary<string, YieldCurve> LockedInYieldCurve { get; private set; }
    public Dictionary<string, Dictionary<int, YieldCurve>> CurrentYieldCurve { get; private set; }
    // public Dictionary<string, PartnerRating> LockedInPartnerRating { get; private set; }
    public Dictionary<int, Dictionary<string, PartnerRating>> LockedInPartnerRating { get; private set; }
    public Dictionary<string, Dictionary<int, PartnerRating>> CurrentPartnerRating { get; private set; }
    // public Dictionary<string, CreditDefaultRate> LockedInCreditDefaultRates { get; private set; }
    public Dictionary<int, Dictionary<string, CreditDefaultRate>> LockedInCreditDefaultRates { get; private set; }
    public Dictionary<string, Dictionary<int, CreditDefaultRate>> CurrentCreditDefaultRates { get; private set; }
    public Dictionary<string, Dictionary<int, SingleDataNodeParameter>> SingleDataNodeParametersByGoc { get; private set; }
    public Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>> InterDataNodeParametersByGoc { get; private set; }
    public Dictionary<AocStep, AocConfiguration> AocConfigurationByAocStep { get; private set; }
    
    private Dictionary<InputSource, HashSet<AocStep>> aocStepByInputSource;
    
    //Dimensions
    public Dictionary<string, AmountType> AmountTypeDimension { get; private set; }
    public Dictionary<string, Novelty> NoveltyDimension { get; private set; }
    public Dictionary<string, EstimateType> EstimateTypeDimension { get; private set; }
    public Dictionary<string, HashSet<string>> EstimateTypesByImportFormat { get; private set; }
    public Dictionary<string, string> ExperienceAdjustEstimateTypeMapping { get; private set; } 
    public HashSet<string> ImportActualEstimateTypes => GetImportActualEstimateType();

    //Constructor
    public ImportStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    //Initialize
    public async Task InitializeAsync()
    {   
        //Dimensions
        var estimateTypes = await workspace.Query<EstimateType>().ToArrayAsync();
        
        EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}}; //TODO move this logic
        
        //Hierarchy Cache
        await hierarchyCache.InitializeAsync<AmountType>();
        
        //EstimateType to load and to update
        EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow }
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % periodicityInMonths == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        PartitionByRn = (await querySource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        TargetPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        DefaultPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Set Partitions
        await querySource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
        await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        //Get data from Workspace (result of parsing)
        var parsedRawVariables = await workspace.QueryPartitionedDataAsync<RawVariable,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, ImportFormat);
        var parsedIfrsVariables = await workspace.QueryPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, ImportFormat);
        
        //DataNodes
        DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);
        
        //Accident Years
        AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
            (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
            .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
        // Import Scopes and Data Node relationship parameters
        InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
        
        var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
        var primaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();

        var primaryScope = primaryScopeFromParsedVariables.Concat(primaryScopeFromLinkedReinsurance).ToHashSet();
        var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
        var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
        DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
        // Parameters
        CurrentPartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
        var initialYear = DataNodeDataBySystemName.Values.Select(dn => dn.Year).ToHashSet();
        // LockedInPartnerRating = initialYear.ToDictionary( x => x,
        //                                                      async x => await workspace.LoadCurrentParameterAsync<PartnerRating>(args with {Year = x, Month = args.Year == x ? args.Month : MonthInAYear},
        //                                                             x => x.Partner) );
        LockedInPartnerRating = new Dictionary<int, Dictionary<string, PartnerRating>>();
        LockedInCreditDefaultRates = new Dictionary<int, Dictionary<string, CreditDefaultRate>>();
        foreach (var year in initialYear)
        {
            // var loadedPr = await workspace.LoadCurrentParameterAsync<PartnerRating>(args with { Year = year, Month = args.Year == year ? args.Month : MonthInAYear },
            //     x => x.Partner);
            // var loadedCdr = await workspace.LoadCurrentParameterAsync<CreditDefaultRate>(args with { Year = year, Month = args.Year == year ? args.Month : MonthInAYear },
            //     x => x.CreditRiskRating);
            
            LockedInPartnerRating[year] = await workspace.LoadCurrentParameterAsync<PartnerRating>(args with { Year = year, Month = args.Year == year ? args.Month : MonthInAYear },
                x => x.Partner);
            LockedInCreditDefaultRates[year] = await workspace.LoadCurrentParameterAsync<CreditDefaultRate>(args with { Year = year, Month = args.Year == year ? args.Month : MonthInAYear },
                x => x.CreditRiskRating);
        }
        CurrentCreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
        // var initialYear = DataNodeDataBySystemName.Values.Select(dn => dn.Year).ToHashSet();
        // foreach (var x in initialYear)
        // {
        //     // LockedInPartnerRating = await workspace.LoadCurrentParameterAsync<PartnerRating>(args with { Year = x, Month = args.Year == x ? args.Month : MonthInAYear }, x => x.Partner);
        //     // LockedInCreditDefaultRates = await workspace.LoadCurrentParameterAsync<CreditDefaultRate>(args with { Year = x, Month = args.Year == x ? args.Month : MonthInAYear }, x => x.CreditRiskRating);
        //     var loadedDataPartnerRating = await workspace.LoadCurrentParameterAsync<PartnerRating>(args with { Year = x, Month = args.Year == x ? args.Month : MonthInAYear }, x => x.Partner);
        //     var loadedDataCreditDefaultRate = await workspace.LoadCurrentParameterAsync<CreditDefaultRate>(args with { Year = x, Month = args.Year == x ? args.Month : MonthInAYear }, x => x.CreditRiskRating);
            
        //     foreach (var partnerRating in loadedDataPartnerRating) LockedInPartnerRating[partnerRating.Key] = partnerRating.Value;
        //     foreach (var creditDefaultRate in loadedDataCreditDefaultRate) LockedInCreditDefaultRates[creditDefaultRate.Key] = creditDefaultRate.Value;

        // }
        SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);
        LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
        CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
        AocConfigurationByAocStep = await querySource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
        aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
        //Previous Period
        var openingRawVariables = Enumerable.Empty<RawVariable>();
        var openingIfrsVariables = Enumerable.Empty<IfrsVariable>();

        var allImportScopesAtInceptionYear = allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).Where(dnd => dnd.Year == args.Year).Select(x => x.DataNode).ToHashSet();
        var allImportScopesNotAtInceptionYear = allImportScopes.Except(allImportScopesAtInceptionYear).ToHashSet();

        if(allImportScopesAtInceptionYear.Any()) {
            var primaryScopeAtInceptionYear = allImportScopesAtInceptionYear.Where(dn => primaryScopeFromParsedVariables.Contains(dn));
            var secondaryScopeAtInceptionYear = allImportScopesAtInceptionYear.Where(dn => primaryScopeFromLinkedReinsurance.Contains(dn) || secondaryScope.Contains(dn));

            openingIfrsVariables = await querySource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == TargetPartition && iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I)
                                    .Where(iv => primaryScopeAtInceptionYear.Contains(iv.DataNode) && ImportFormat != ImportFormats.Opening
                                                 && EstimateTypesByImportFormat[InputSource.Opening.ToString()].Contains(iv.EstimateType) 
                                                 || secondaryScopeAtInceptionYear.Contains(iv.DataNode)).ToArrayAsync();
        }

        if(allImportScopesNotAtInceptionYear.Any()) {
            PreviousPeriodPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>()
                                        .Where(p => p.ReportingNode == args.ReportingNode && p.Year == PreviousReportingPeriod.Year 
                                                 && p.Month == PreviousReportingPeriod.Month && p.Scenario == null).ToArrayAsync()).Single().Id;
            
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(PreviousPeriodPartition);
        
            //Perform queries to previous Period
            openingRawVariables = (await querySource.Query<RawVariable>()
                                   .Where(rv => rv.Partition == PreviousPeriodPartition && rv.AocType == AocTypes.CL)
                                   .Where(v => primaryScope.Contains(v.DataNode)).ToArrayAsync())
                                   .Select(rv => rv with {AocType = AocTypes.BOP, Novelty = Novelties.I, 
                                                 Values = rv.Values.Skip(MonthInAYear).ToArray(), Partition = TargetPartition});
            
            openingIfrsVariables = openingIfrsVariables.Union((await querySource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == PreviousPeriodPartition && iv.AocType == AocTypes.EOP)
                                    .Where(v => allImportScopesNotAtInceptionYear.Contains(v.DataNode)).ToArrayAsync())
                                    .Select(iv => iv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Partition = TargetPartition}),
                                    EqualityComparer<IfrsVariable>.Instance);
            
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);

            // TODO: print error if 
            //openingRawVariables.Select(x => x.DataNode).ToHashSet() != dataNodesWithPreviousPeriod
        }
        
        //Variables
        var rawVariables = parsedRawVariables.Concat(openingRawVariables)
                                             .Concat(await querySource.Query<RawVariable>()
                                                                      .Where(rv => rv.Partition == TargetPartition)
                                                                      .Where(rv => primaryScopeFromLinkedReinsurance.Contains(rv.DataNode)).ToArrayAsync()); 
        
        var ifrsVariables = parsedIfrsVariables.Union(openingIfrsVariables, EqualityComparer<IfrsVariable>.Instance)
                                               .Concat(await querySource.Query<IfrsVariable>()
                                                                        .Where(iv => iv.Partition == TargetPartition && !(iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I))
                                                                        .Where(iv => primaryScopeFromParsedVariables.Contains(iv.DataNode) 
                                                                                     && !EstimateTypesByImportFormat[ImportFormat].Contains(iv.EstimateType)  
                                                                                     || primaryScopeFromLinkedReinsurance.Contains(iv.DataNode) 
                                                                                     || secondaryScope.Contains(iv.DataNode)).ToArrayAsync());                                                                       

        if(DefaultPartition != TargetPartition) {
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(DefaultPartition);
            var defaultRawVariables = await querySource.Query<RawVariable>().Where(rv => rv.Partition == DefaultPartition && primaryScope.Contains(rv.DataNode)).ToArrayAsync();
            var defaultIfrsVariables = await querySource.Query<IfrsVariable>().Where(iv => iv.Partition == DefaultPartition && allImportScopes.Contains(iv.DataNode)).ToArrayAsync();         
            rawVariables = rawVariables.Union(defaultRawVariables, EqualityComparer<RawVariable>.Instance);
            ifrsVariables = ifrsVariables.Union(defaultIfrsVariables, EqualityComparer<IfrsVariable>.Instance);
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        }

        RawVariablesByImportIdentity = (IDictionary<string, ICollection<RawVariable>>)rawVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<RawVariable>)v.ToArray());
        IfrsVariablesByImportIdentity = (IDictionary<string, ICollection<IfrsVariable>>)ifrsVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<IfrsVariable>)v.ToArray());
    }
    
    //Getters
    
    //Periods
    public ValuationPeriod GetValuationPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].ValuationPeriod;
    public PeriodType GetYieldCurvePeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].YcPeriod;
    public PeriodType GetCreditDefaultRiskPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].CdrPeriod;
    
    public IEnumerable<AocStep> GetAllAocSteps(InputSource source) => aocStepByInputSource[source];
    public IEnumerable<AocStep> GetCalculatedTelescopicAocSteps() => AocConfigurationByAocStep.Where(kvp => kvp.Value.DataType == DataType.CalculatedTelescopic).Select(kvp => kvp.Key);

    //YieldCurve
    public double[] GetYearlyYieldCurve(ImportIdentity id, string economicBasis) {
        var yc = GetYieldCurve(id, economicBasis);
        return yc.Values.Skip(args.Year - yc.Year).ToArray(); //Check if the returned array is empty? Log Warning?
    }
    
    public YieldCurve GetYieldCurve(ImportIdentity id, string economicBasis) => (economicBasis, GetYieldCurvePeriod(id)) switch {
            (EconomicBases.C, PeriodType.BeginningOfPeriod ) => GetShift(id.ProjectionPeriod) > 0 
                                     ? CurrentYieldCurve[id.DataNode][CurrentPeriod]
                                     : CurrentYieldCurve[id.DataNode][PreviousPeriod],
            (EconomicBases.C, PeriodType.EndOfPeriod) => CurrentYieldCurve[id.DataNode][CurrentPeriod],            
            (EconomicBases.L, _ ) => LockedInYieldCurve[id.DataNode],
            (_, PeriodType.NotApplicable) => (YieldCurve)ApplicationMessage.Log(Error.YieldCurvePeriodNotApplicable, id.AocType, id.Novelty),
            (_, _) => (YieldCurve)ApplicationMessage.Log(Error.EconomicBasisNotFound, id.DataNode)
       };

    //int Identity.ProjectionPeriod 
    public int GetProjectionCount() => ProjectionConfiguration.Count();
    public int GetShift(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].Shift;
    public int GetTimeStep(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].TimeStep;
    
    public PeriodType GetPeriodType(string amountType, string estimateType) => 
                    amountType != null && AmountTypeDimension.TryGetValue(amountType, out var at) 
                                ? at.PeriodType
                                : estimateType != null && EstimateTypeDimension.TryGetValue(estimateType, out var ct) 
                                    ? ct.PeriodType : PeriodType.EndOfPeriod;

    //Variables and Cash flows
    
    public IEnumerable<RawVariable> GetRawVariables(string dataNode) => RawVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<RawVariable>();
    public IEnumerable<IfrsVariable> GetIfrsVariables(string dataNode) => IfrsVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<IfrsVariable>();
    
    public double[] GetValues(ImportIdentity id, Func<RawVariable, bool> whereClause) => GetRawVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Select(v => v?.Values ?? (double[])null).AggregateDoubleArray();
    public double GetValue(ImportIdentity id, Func<IfrsVariable, bool> whereClause, int projection = 0) => GetIfrsVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Select(v => v?.Values ?? (double[])null).AggregateDoubleArray().ElementAtOrDefault(projection);
    
    public double[] GetValues(ImportIdentity id, string amountType, string estimateType, int? accidentYear) => GetValues(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType);
    public double GetValue(ImportIdentity id, string amountType, string estimateType, int? accidentYear, int projection = 0) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType, projection);
    public double GetValue(ImportIdentity id, string amountType, string estimateType, string economicBasis, int? accidentYear, int projection = 0) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType && v.EconomicBasis == economicBasis, projection);
   
    //Novelty
    private IEnumerable<string> GetNoveltiesForAocType(string aocType, IEnumerable<AocStep> aocConfiguration) => aocConfiguration.Where(aocStep => aocStep.AocType == aocType).Select(aocStep => aocStep.Novelty);
    public IEnumerable<string> GetNovelties() => NoveltyDimension.Keys;
    public IEnumerable<string> GetNovelties(string aocType) => GetNoveltiesForAocType(aocType, AocConfigurationByAocStep.Keys);
    public IEnumerable<string> GetNovelties(string aocType, InputSource inputSource) => GetNoveltiesForAocType(aocType, aocStepByInputSource[inputSource]);
    
    //Accident years
    public IEnumerable<int?> GetAccidentYears(string dataNode) => AccidentYearsByDataNode.TryGetValue(dataNode, out var accidentYear) ? accidentYear : new int?[] { null };
    
    //Parameters
    public double GetLockedInNonPerformanceRiskRate(ImportIdentity identity)
    {
        if(!DataNodeDataBySystemName.TryGetValue(identity.DataNode, out var dataNodeData))     ApplicationMessage.Log(Error.DataNodeNotFound, identity.DataNode);
        if(dataNodeData.Partner == null)                                                       ApplicationMessage.Log(Error.PartnerNotFound, identity.DataNode);
    
        if(!LockedInPartnerRating[dataNodeData.Year].TryGetValue(dataNodeData.Partner, out var rating))         ApplicationMessage.Log(Error.RatingNotFound, dataNodeData.Partner);
        if(!LockedInCreditDefaultRates[dataNodeData.Year].TryGetValue(rating.CreditRiskRating, out var rate))   ApplicationMessage.Log(Error.CreditDefaultRateNotFound, rating.CreditRiskRating);
        return Math.Pow(1d + rate.Values[0], 1d / 12d) - 1d;
    }

    public double GetCurrentNonPerformanceRiskRate (ImportIdentity identity) {
        var period = GetCreditDefaultRiskPeriod(identity) == PeriodType.BeginningOfPeriod ? PreviousPeriod : CurrentPeriod;
       
        if(!DataNodeDataBySystemName.TryGetValue(identity.DataNode, out var dataNodeData))     ApplicationMessage.Log(Error.DataNodeNotFound, identity.DataNode);
        if(dataNodeData.Partner == null)                                                       ApplicationMessage.Log(Error.PartnerNotFound, identity.DataNode);
        // if Partner == Internal then return 0;
        if(!CurrentPartnerRating.TryGetValue(dataNodeData.Partner, out var rating))                   ApplicationMessage.Log(Error.RatingNotFound, dataNodeData.Partner);
        if(!CurrentCreditDefaultRates.TryGetValue(rating[period].CreditRiskRating, out var rate))     ApplicationMessage.Log(Error.CreditDefaultRateNotFound, rating[period].CreditRiskRating);
        return Math.Pow(1d + rate[period].Values[0], 1d / 12d) - 1d;
    }

    public double GetNonPerformanceRiskRate (ImportIdentity identity, string AmountType)
    {
        if(AmountType == AmountTypes.CDR) 
            return GetCurrentNonPerformanceRiskRate(identity);
        return GetLockedInNonPerformanceRiskRate(identity);
    }
    
    public double GetPremiumAllocationFactor(ImportIdentity id) => SingleDataNodeParametersByGoc.TryGetValue(id.DataNode, out var singleDataNodeParameter) 
                                                                                ? singleDataNodeParameter[CurrentPeriod].PremiumAllocation : DefaultPremiumExperienceAdjustmentFactor;
    
    // Data Node relationships
    public IEnumerable<string> GetUnderlyingGic(ImportIdentity id) => !InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
        ? Enumerable.Empty<string>()
        : interDataNodeParameters[CurrentPeriod].Select(x => x.DataNode != id.DataNode ? x.DataNode : x.LinkedDataNode).Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance);
    
    public double GetReinsuranceCoverage (ImportIdentity id, string gic)  
    {
        var targetPeriod = AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].RcPeriod == PeriodType.EndOfPeriod ? CurrentPeriod : PreviousPeriod;
        return InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
            ? interDataNodeParameters[targetPeriod].FirstOrDefault(x => x.DataNode == gic || x.LinkedDataNode == gic).ReinsuranceCoverage
            : (double)ApplicationMessage.Log(Error.ReinsuranceCoverage, id.DataNode);
    }
    
    // Import Scope
    public bool IsPrimaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Primary].Contains(dataNode);
    public bool IsSecondaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Secondary].Contains(dataNode);
    
    // Other
    public IEnumerable<string> GetNonAttributableAmountType() => new string[]{AmountTypes.NE};
    public IEnumerable<string> GetAttributableExpenseAndCommissionAmountType() => hierarchyCache.Get<AmountType>(AmountTypes.ACA).Descendants(includeSelf : true).Select(x => x.SystemName)
                                                                                   .Concat(hierarchyCache.Get<AmountType>(AmountTypes.AEA).Descendants(includeSelf : true).Select(x => x.SystemName));
    public IEnumerable<string> GetInvestmentClaims() => hierarchyCache.Get<AmountType>(AmountTypes.ICO).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetPremiums() => hierarchyCache.Get<AmountType>(AmountTypes.PR).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetClaims() => hierarchyCache.Get<AmountType>(AmountTypes.CL).Descendants().Select(x => x.SystemName);
    public IEnumerable<string> GetCdr() => hierarchyCache.Get<AmountType>(AmountTypes.CDR).Descendants(includeSelf: true).Select(x => x.SystemName);
}